In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
!pip install imblearn

In [4]:
import imblearn

ModuleNotFoundError: No module named 'imblearn'

In [2]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sonih\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = pd.read_csv("finalDataSet.csv")
dataset.head()

,Unnamed: 0,Review Text,Polarity_Rating
0,0,Amazing relaxed safe trip services good staff ...,Positive
1,1,great time went 1st marriage anniversary great...,Positive
2,2,normally stay Taj Lands End due location Bandr...,Positive
3,5,hotel pretty good amazingly located Quite easy...,Positive
4,6,Wonderful experience staff excellent polite ev...,Positive


In [4]:
dataset.dtypes

Unnamed: 0          int64
Review Text        object
Polarity_Rating    object
dtype: object

In [5]:
dataset.isna().sum()

Unnamed: 0         0
Review Text        0
Polarity_Rating    0
dtype: int64

In [6]:
dataset.drop(['Unnamed: 0'], axis=1, inplace=True)
dataset.head()

,Review Text,Polarity_Rating
0,Amazing relaxed safe trip services good staff ...,Positive
1,great time went 1st marriage anniversary great...,Positive
2,normally stay Taj Lands End due location Bandr...,Positive
3,hotel pretty good amazingly located Quite easy...,Positive
4,Wonderful experience staff excellent polite ev...,Positive


In [7]:
Polarity_Rating = {'Positive':2,'Neutral':1,'Negative':0}
dataset['Polarity_Rating'] = dataset['Polarity_Rating'].map(Polarity_Rating)
dataset.head()

,Review Text,Polarity_Rating
0,Amazing relaxed safe trip services good staff ...,2
1,great time went 1st marriage anniversary great...,2
2,normally stay Taj Lands End due location Bandr...,2
3,hotel pretty good amazingly located Quite easy...,2
4,Wonderful experience staff excellent polite ev...,2


In [8]:
dataset.shape

(2400, 2)

In [9]:
dataset['Review Text'].values[0]

'Amazing relaxed safe trip services good staff cooperative especially Masala bay reservations guy took extra effort could get best deal'

In [10]:
X = dataset["Review Text"].values
y = dataset.drop("Review Text", axis=1).values
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42 )

## vectorize the data

In [11]:
max_len = -1
for ele in range(len(X)): 
    if len(X[ele]) > max_len: 
        max_len = len(X[ele]) 
        res = ele 

In [12]:
res

2

In [13]:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [14]:
X_train[1]

<1x6788 sparse matrix of type '<class 'numpy.int64'>'
	with 61 stored elements in Compressed Sparse Row format>

In [15]:
X_train[res].shape[1]

6788

In [16]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [17]:
X_train.shape

(1440, 6217)

In [18]:
X_test.shape

(360, 6217)

In [19]:
X_train[res].shape[0]

6217

## use machine learning algorithms for training

# 1. Random Forest Algorithm

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf_cl = RandomForestClassifier()

In [17]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
import numpy as np
## Randomized Search CV

# Number of trees in random forest
n_estimators = [int(i) for i in np.linspace(start=100, stop=1200, num=12)]

# Number of features to consider at evry split
max_features = ['auto', 'sqrt']

# max number of levels in tree
max_depth = [int(i) for i in np.linspace(5, 30, num=6)]

# max_depth.append(None)
# min number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]

#min number of sample required for each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [19]:
# Create the random_grid 

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10]}


In [14]:
rf_random = RandomizedSearchCV(estimator=rf_cl, param_distributions=random_grid, scoring='neg_mean_squared_error', n_iter=10, cv=5,verbose=2, random_state=42, n_jobs=1)

In [44]:
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=900, min_samples_split=5, min_samples_leaf=5, max_features=sqrt, max_depth=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\users\sonih\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=900, min_samples_split=5, min_samples_leaf=5, max_features=sqrt, max_depth=10, total=  13.8s
[CV] n_estimators=900, min_samples_split=5, min_samples_leaf=5, max_features=sqrt, max_depth=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.7s remaining:    0.0s
c:\users\sonih\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=900, min_samples_split=5, min_samples_leaf=5, max_features=sqrt, max_depth=10, total=  13.7s
[CV] n_estimators=900, min_samples_split=5, min_samples_leaf=5, max_features=sqrt, max_depth=10 


...

[CV]  n_estimators=700, min_samples_split=15, min_samples_leaf=1, max_features=auto, max_depth=20, total=  16.3s
[CV] n_estimators=700, min_samples_split=15, min_samples_leaf=1, max_features=auto, max_depth=20 


c:\users\sonih\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=700, min_samples_split=15, min_samples_leaf=1, max_features=auto, max_depth=20, total=  16.9s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 10.8min finished
c:\users\sonih\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [48]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 25}

In [20]:
best_random_forest_model = RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=25)

In [21]:
best_random_forest_model.fit(X_train, y_train)

c:\users\sonih\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_depth=25, max_features='sqrt', n_estimators=1000)

In [22]:
pred = best_random_forest_model.predict(X_test)

In [23]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [24]:
accuracy_score(y_test, pred)

0.9604166666666667

In [25]:
from sklearn.metrics import classification_report, confusion_matrix

In [26]:
print(confusion_matrix(y_test, pred))

[[155   0   2]
 [  4 146   5]
 [  4   4 160]]


In [27]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       157
           1       0.97      0.94      0.96       155
           2       0.96      0.95      0.96       168

    accuracy                           0.96       480
   macro avg       0.96      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



In [52]:
review = ["staff behaviuor is bad"]

In [53]:
r = vect.transform(review)

In [54]:
r

<1x6788 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [55]:
best_random_forest_model.predict(r)

array([0], dtype=int64)

In [57]:
import pickle



In [58]:
file = open('vectorize.pkl','wb')

# dump information to that file
pickle.dump(vect, file)
file.close()

In [59]:
# open a file, where you want to share the data
file = open('randomForest_clssify_model.pkl','wb')

# dump information to that file
pickle.dump(best_random_forest_model, file)
file.close()